<a href="https://colab.research.google.com/github/EastHuni/lg-aimers/blob/main/%5BBaseline%5D_lag%2Ctrend_and_so_on_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import glob
import re
import holidays

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm


# Fixed RandomSeed & Setting Hyperparameter

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Load

In [ ]:
import zipfile
import os


zip_path = '/content/drive/MyDrive/LG/open.zip'
extract_path = '/content/LG_data'
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
train = pd.read_csv('/content/LG_data/train/train.csv')
train

,영업일자,영업장명_메뉴명,매출수량
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0
...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12
102672,2024-06-12,화담숲카페_현미뻥스크림,10
102673,2024-06-13,화담숲카페_현미뻥스크림,14
102674,2024-06-14,화담숲카페_현미뻥스크림,12


# Define Model

In [ ]:
pip install holidays

In [ ]:
def make_features(df):
    df = df.sort_values(['영업장명_메뉴명', '영업일자']).copy()
    df['영업일자'] = pd.to_datetime(df['영업일자'])

    df['요일'] = df['영업일자'].dt.dayofweek

    df['lag_1'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(1)
    df['lag_3'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(3)
    df['lag_7'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(7)

    df['rolling_7_mean'] = (
        df.groupby('영업장명_메뉴명')['매출수량']
        .shift(1).rolling(7, min_periods=1).mean()
        .reset_index(0, drop=True)
    )

    df['trend_7'] = (
        df.groupby('영업장명_메뉴명')['매출수량']
        .shift(1).rolling(7, min_periods=1)
        .apply(lambda x: x[-1] - x[0], raw=True)
        .reset_index(0, drop=True)
    )

    feature_cols = ['lag_1', 'lag_3', 'lag_7', 'rolling_7_mean', 'trend_7']
    df[feature_cols] = df.groupby('영업장명_메뉴명')[feature_cols].ffill()
    rolling_avg = (
        df.groupby('영업장명_메뉴명')['매출수량']
        .transform(lambda x: x.shift(1).rolling(7, min_periods=1).mean())
    )

    for col in feature_cols:
        df[col] = df[col].fillna(rolling_avg)

    df[feature_cols] = df[feature_cols].fillna(0)

    selected_cols = ['영업일자', '영업장명_메뉴명', '매출수량', '요일'] + feature_cols
    return df[selected_cols]


In [ ]:
train = make_features(train).copy()

In [ ]:
train

,영업일자,영업장명_메뉴명,매출수량,요일,lag_1,lag_3,lag_7,rolling_7_mean,trend_7
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,6,0.0,0.0,0.0,0.000000,0.0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,0,0.0,0.0,0.0,0.000000,0.0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,1,0.0,0.0,0.0,0.000000,0.0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2,0.0,0.0,0.0,0.000000,0.0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,3,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,1,0.0,25.0,25.0,44.857143,-25.0
102672,2024-06-12,화담숲카페_현미뻥스크림,10,2,12.0,43.0,16.0,43.000000,-4.0
102673,2024-06-13,화담숲카페_현미뻥스크림,14,3,10.0,0.0,145.0,42.142857,-135.0
102674,2024-06-14,화담숲카페_현미뻥스크림,12,4,14.0,12.0,60.0,23.428571,-46.0


In [ ]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])


def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량', '요일',	'lag_1',	'lag_3',	'lag_7',	'rolling_7_mean',	'trend_7']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(X_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=7, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models
# 학습
trained_models = train_lstm(train)
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted[['매출수량', '요일',	'lag_1',	'lag_3',	'lag_7',	'rolling_7_mean',	'trend_7']].values[-LOOKBACK:]  # shape: (28, 3)
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals)
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 7))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val

            })

    return pd.DataFrame(results)

Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]/tmp/ipython-input-2851374203.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X_train = torch.tensor(X_train).float().to(DEVICE)
Training LSTM: 100%|██████████| 193/193 [13:13<00:00,  4.11s/it]


# Train

# Prediction

In [ ]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted[['매출수량', '요일',	'lag_1', 'lag_3', 'lag_7', 'rolling_7_mean', 'trend_7']].values[-LOOKBACK:]  # shape: (28, 3)
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals)
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 7))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val

            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('/content/LG_data/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)
    test_df = make_features(test_df)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/valida

In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
      zip(pred_df['영업일자'], pred_df['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)),
      pred_df['매출수량']
      ))


    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for cols in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, cols] = pred_dict.get((date, cols,), 0)

    return final_df


In [ ]:
sample_submission = pd.read_csv('/content/LG_data/sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')


/tmp/ipython-input-3115272524.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.438545390963554' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, cols] = pred_dict.get((date, cols,), 0)
/tmp/ipython-input-3115272524.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.779537320137024' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, cols] = pred_dict.get((date, cols,), 0)
/tmp/ipython-input-3115272524.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.1392399966716766' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, cols] = pred_dict.get((date, co

# Submission